In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf

/Users/christinearnoldt/mambaforge/envs/iannwtf/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
BATCH_SIZE = 8

In [ ]:
# FEHLERHAFT


ds = tfds.load("fashion_mnist", split="train", as_supervised=True)
ds = ds.map(lambda img, target: (tf.reshape(img, (-1,)), target))
df = ds.map(lambda img, target: (tf.cast(img, tf.float32), target))
ds = ds.map(lambda img, target: ((img/128.)-1, tf.one_hot(target, depth=10)))
ds = ds.shuffle(1000).batch(BATCH_SIZE).prefetch(2)


In [ ]:
class QuickFashionMNISTMLP(tf.keras.Model):
    def __init__(self, layer_sizes):
        super().__init__()
        self.hidden_layers = [tf.keras.layers.Dense(ls, activation='relu') for ls in layer_sizes]
        self.out_layer = tf.keras.layers.Dense(10, activation='softmax')
    def call(self,x,training=True):
        for layer in self.hidden_layers:
            x = layer(x)
        y = self.out_layer(x)
        return y

In [ ]:
def get_model_from_sequential(layer_sizes):
    model = tf.keras.Sequential()
    model.add(tf.keras.Input(shape=(784,), dtype=tf.float32))
    
    for ls in layer_sizes:
        model.add(tf.keras.layers.Dense(ls, activation="relu"))
    model.add(tf.keras.layers.Dense(10,activation="softmax"))
    return model

def quick_get_model_from_sequential(layer_sizes):
    model = tf.keras.Sequential([[tf.keras.layers.Dense(ls, activation='relu') for ls in layer_sizes]+[tf.keras.layers.Dense(10,activation="softmax")]])
    return model

In [ ]:
def get_model_from_functional(layer_sizes, name):
    inputs = tf.keras.Input(shape=(784,), dtype=tf.float32)
    x = inputs
    for ls in layer_sizes:
        x = tf.keras.layers.Dense(ls, activation="relu")(x)
    y = tf.keras.layers.Dense(10, activation="softmax")(x)
    model = tf.keras.Model(inputs=inputs, outputs=y, name=name)
    return model